In [12]:
!hostnamectl
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
   Static hostname: c206-1
         Icon name: computer-server
           Chassis: server
        Machine ID: d83a25f1a3734753bf847d7ccec7293d
           Boot ID: 0819a9eb62dc48a5b14fe8af8405b988
  Operating System: ]8;;https://rockylinux.org/Rocky Linux 8.4 (Green Obsidian)]8;;
       CPE OS Name: cpe:/o:rocky:rocky:8.4:GA
            Kernel: Linux 4.18.0-305.3.1.el8_4.x86_64
      Architecture: x86-64
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the 

### La librería Transformers

La forma más sencilla de experimentar con la librería **Transformers** es a través de la función *pipeline*. Ésta permite ejecutar la tarea de inferencia de un modelo


In [47]:
from transformers import pipeline
import gradio as gr

# Crear el clasificador de análisis de sentimientos
classifier = pipeline('sentiment-analysis')
print(classifier('I love you')[0]['label'])
 

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


POSITIVE


También podemos procesar más de una entrada

In [48]:
results = classifier(["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."])
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

label: POSITIVE, with score: 0.9998
label: NEGATIVE, with score: 0.5309


También podemos procesar un *dataset* completo 

In [49]:
%pip install soundfile librosa
import torch
from transformers import pipeline
speech_recognizer = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-base-960h")
from datasets import load_dataset, Audio
dataset = load_dataset("PolyAI/minds14", name="en-US", split="train")
dataset = dataset.cast_column("audio", Audio(sampling_rate=speech_recognizer.feature_extractor.sampling_rate))
result = speech_recognizer(dataset[:4]["audio"])
print([d["text"] for d in result])

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Found cached dataset minds14 (/home/ulc/es/dac/.cache/huggingface/datasets/PolyAI___minds14/en-US/1.0.0/7bdf27fccad4efc6da7c00f739d96bfa0d25a7a85f2244755a73c865f70182bc)


['I WOULD LIKE TO SET UP A JOINT ACCOUNT WITH MY PARTNER HOW DO I PROCEED WITH DOING THAT', "FONDERING HOW I'D SET UP A JOIN TO HELL T WITH MY WIFE AND WHERE THE AP MIGHT BE", "I I'D LIKE TOY SET UP A JOINT ACCOUNT WITH MY PARTNER I'M NOT SEEING THE OPTION TO DO IT ON THE APSO I CALLED IN TO GET SOME HELP CAN I JUST DO IT OVER THE PHONE WITH YOU AND GIVE YOU THE INFORMATION OR SHOULD I DO IT IN THE AP AN I'M MISSING SOMETHING UQUETTE HAD PREFERRED TO JUST DO IT OVER THE PHONE OF POSSIBLE THINGS", 'HOW DO I FURN A JOINA COUT']


## Autoclasses

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AutoTokenizer

model_name= "nlptown/bert-base-multilingual-uncased-sentiment"
model=AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
encoding=tokenizer("I love you")
print(encoding)
pt_batch = tokenizer(["I love you", "I hate you"], padding=True, truncation=True, return_tensors="pt")
pt_outputs = model(**pt_batch)  
from torch import nn
pt_predictions = nn.functional.softmax(pt_outputs.logits, dim=-1)
print(pt_predictions)

{'input_ids': [101, 1045, 2293, 2017, 102], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}
tensor([[0.1631, 0.1457, 0.2296, 0.2280, 0.2336],
        [0.1635, 0.1629, 0.2798, 0.2095, 0.1843]], grad_fn=<SoftmaxBackward0>)


## Entrenamiento

In [9]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

from datasets import load_dataset
dataset = load_dataset("rotten_tomatoes")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

dataset = dataset.map(tokenize_function, batched=True)

from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer)

from transformers import Trainer
trainer=Trainer(
    model=model,
    data_collator=data_collator,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
)
trainer.train()

Found cached dataset parquet (/home/ulc/es/dac/.cache/huggingface/datasets/parquet/rotten_tomatoes-ed76130f42b6916d/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/ulc/es/dac/.cache/huggingface/datasets/parquet/rotten_tomatoes-ed76130f42b6916d/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-1e7383e71a39a15b.arrow
Loading cached processed dataset at /home/ulc/es/dac/.cache/huggingface/datasets/parquet/rotten_tomatoes-ed76130f42b6916d/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-43ffd9d34d430d55.arrow
Loading cached processed dataset at /home/ulc/es/dac/.cache/huggingface/datasets/parquet/rotten_tomatoes-ed76130f42b6916d/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-4aece9c6dff4a408.arrow
/mnt/netapp2/Store_uni/home/ulc/es/dac/mytx/lib/python3.7/site-packages/transformers/optimization.py:415: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWa

Step,Training Loss
500,0.717400


KeyboardInterrupt: 